# 변동성 타게팅(Volatility Targeting)

|레벨|스타일|기대CAGR|매수전략|매도전략|
|:--:|:--:|:--:|:--:|:--:|
|중급|자산배분|변동성에 따라 다름|- 특정 주가지수, 주식 또는 포트폴리오를 선택<br>- 주가지수, 주식 또는 포트폴리오의 변동성을 계산<br>- 투자 비중을 목표 변동성에 맞춤<br>- 평균 모멘텀 스코어를 통해 다시 한 번 투자 비중을 조정<br>- 나머지 금액은 현금 보유|월 1회 리밸런싱|

In [ ]:
# Target_vol / port_vol
# 100000 * ( 1.0 / 0.5 low)high

# Load Libraries & Dataset

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

In [ ]:
kodex_200 = pd.DataFrame()
kodex_url = 'https://finance.naver.com/item/sise_day.nhn?code=069500' 
for page in tqdm(range(1, 100)):
    pg_url = '{url}&page={page}'.format(url=kodex_url, page=page)
    pg_url = requests.get(pg_url, headers={'User-agent' : 'Mozilla/5.0'}).text
    kodex_200 = kodex_200.append(pd.read_html(pg_url)[0])

kodex_200 = kodex_200.dropna()
kodex_200.columns = ['date', 'close', 'over_the_day', 'open', 'high', 'low', 'volume']
kodex_200.date = pd.to_datetime(kodex_200.date)

In [ ]:
kodex_200 = kodex_200.sort_values(by='date')
kodex_200.index = [x for x in range(len(kodex_200))]
kodex_200

#Convert into Monthly data

In [ ]:
kodex_monthly = pd.DataFrame()
for y in range(2017, 2022):
    for m in range(1, 13):
        kodex_monthly = kodex_monthly.append(
            kodex_200[(kodex_200['date'].dt.year == y) & (kodex_200['date'].dt.month == m)].head(1).head(1))

kodex_monthly.index = [x for x in range(len(kodex_monthly))]

kodex_monthly

In [ ]:
len(kodex_monthly)

#Calculate Volatility(Standard deviation)

In [ ]:
std_list = []

for y in range(2017, 2022):
    for m in range(1, 13):
        abs_std = kodex_200[(kodex_200.date.dt.year == y) & (kodex_200.date.dt.month == m)].std().close
        monthly_mean = kodex_200[(kodex_200.date.dt.year == y) & (kodex_200.date.dt.month == m)].mean().close
        rel_std = abs_std / monthly_mean * 100


        if rel_std > 0:
            std_list.append(rel_std)

len(std_list)



In [ ]:
len(kodex_monthly)

In [ ]:
kodex_monthly['std'] = std_list
kodex_monthly.head()

# Calculate RoR

In [ ]:
ror_list = []

for i in range(len(kodex_monthly)):
    try:

        ror = kodex_monthly.close.diff().iloc[i] /  kodex_monthly.close.iloc[i-1] * 100
    except:
        ror = 0 
    ror_list.append(ror)
len(ror_list)




In [ ]:
kodex_monthly['RoR'] = ror_list
kodex_monthly = kodex_monthly.fillna(0)
#kodex_monthly 


In [ ]:
kodex_monthly['Plus'] = kodex_monthly.RoR > 0
kodex_monthly.head(10)

In [ ]:
kodex_monthly['momentum_score'] = kodex_monthly.Plus.rolling(window=12).sum()
kodex_monthly.head(20)

In [ ]:
kodex_monthly = kodex_monthly.fillna(0)


In [ ]:
kodex_monthly = kodex_monthly[11:]
kodex_monthly

$ StockPct = (TVol/PVol) * (MomentumScore/12) $
$ TVol = TargetValue, PVol = PortfolioValue $

#Backtest

In [ ]:
# pseudo code
def backtest(beginning_asset=100000, target_vol = 1):
    asset = [beginning_asset]
    
    for i in range(len(kodex_monthly)-1):
        stock_pct = (target_vol/kodex_monthly.iloc[i]['std'] )* (kodex_monthly.iloc[i].momentum_score/12)
        if stock_pct > 1:
            stock_pct = 1 # 1을 넘어가면 무조건 1로 고정
        #(만약 좌항 값이 1을 넘기면 Max Value는 1)
        cash_pct = 1 - stock_pct

        stock_shares = (beginning_asset * stock_pct) / kodex_monthly.iloc[i].close # t 시점
        stcok_ending_value = stock_shares *  kodex_monthly.iloc[i+1].close # t+1 시점

        cash_ending_value = beginning_asset * cash_pct

        total_ending_value = stcok_ending_value + cash_ending_value

        asset.append(total_ending_value) 
        beginning_asset = total_ending_value 





    return asset

In [ ]:
result1 = backtest()
result2 = backtest(target_vol = 2)
result3= backtest(target_vol = 3)
result4 = backtest(target_vol = 4)

In [ ]:
back_test = {
    'date' : kodex_monthly.date,
    'backtest1' : result1,
    'backtest2' : result2,
    'backtest3' : result3,
    'backtest4' : result4,
    'KODEX_200' : kodex_monthly.close / kodex_monthly.iloc[0].close * 100000

}
back_test = pd.DataFrame(back_test)

In [ ]:
back_test.head()

#Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
back_test

In [ ]:
plt.figure(figsize=(18, 4))

plt.plot(back_test.date, back_test.backtest1, label='backtest1')
plt.plot(back_test.date, back_test.backtest2, label='backtest2')
plt.plot(back_test.date, back_test.backtest3, label='backtest3')
plt.plot(back_test.date, back_test.backtest4, label='backtest4')
plt.plot(back_test.date, back_test.KODEX_200, label='KODEX_200')

plt.grid()
plt.legend(loc='best')
plt.title('Backtesting Portfolio & Benchmark', fontsize=25)
plt.xlabel('Year', fontsize=18)
plt.ylabel('Asset Amount', fontsize=18)


plt.show()

#Compute CAGR

In [ ]:
start, end = result1[0], result1[-1]
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1 ) * 100
print('%.2f'%cagr, '%')

In [ ]:
start, end = result2[0], result2[-1]
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1 ) * 100
print('%.2f'%cagr, '%')

In [ ]:
start, end = result3[0], result3[-1]
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1 ) * 100
print('%.2f'%cagr, '%')

In [ ]:
start, end = result4[0], result4[-1]
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1 ) * 100
print('%.2f'%cagr, '%')

In [ ]:
start, end = kodex_monthly.close.iloc[0], kodex_monthly.close.iloc[-1]
cagr = ((end/start)**(1/len(set(kodex_monthly.date.dt.year)))-1 ) * 100
print('%.2f'%cagr, '%')

#Compute Statistics

In [ ]:
# standard deviation of portfolio
import numpy as np
std_pct = (np.std(back_test) / np.mean(back_test)) * 100
std_pct

In [ ]:
# rates of return

for i in range(len(back_test) - 1):
    back_test['RoR1'] = (back_test['backtest1'].diff() / back_test.iloc[i]['backtest1']) * 100
    back_test['RoR2'] = (back_test['backtest2'].diff() / back_test.iloc[i]['backtest2']) * 100
    back_test['RoR3'] = (back_test['backtest3'].diff() / back_test.iloc[i]['backtest3']) * 100
    back_test['RoR4'] = (back_test['backtest4'].diff() / back_test.iloc[i]['backtest4']) * 100
    back_test['kodex_RoR'] = (back_test['KODEX_200'].diff() / back_test.iloc[i]['KODEX_200']) * 100
back_test = back_test.fillna(0)
back_test.head(24)

In [ ]:
# std of RoR

std_ror1 = (np.std(back_test['RoR1']) / np.mean(back_test['RoR1']))
std_ror2 = (np.std(back_test['RoR2']) / np.mean(back_test['RoR2']))
std_ror3 = (np.std(back_test['RoR3']) / np.mean(back_test['RoR3']))
std_ror4 = (np.std(back_test['RoR4']) / np.mean(back_test['RoR4']))
std_kodex_ror = (np.std(back_test['kodex_RoR']) / np.mean(back_test['kodex_RoR']))
print(' std_ror1 :', std_ror1, '\n','std_ror2 :', std_ror2,'\n','std_ror3 :', std_ror3,'\n','std_ror4 :', std_ror4,'\n','kodex_200 :', std_kodex_ror)

In [ ]:
figure, ((ax1, ax2 ,ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3)
figure.set_size_inches(18,4)


sns.distplot(back_test['RoR1'], ax=ax1)
sns.distplot(back_test['RoR2'], ax=ax2)
sns.distplot(back_test['RoR3'], ax=ax3)
sns.distplot(back_test['RoR4'], ax=ax4)
sns.distplot(back_test['kodex_RoR'], ax=ax5)

In [ ]:
figure, (ax1) = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18,4)


sns.distplot(back_test['RoR1'])
sns.distplot(back_test['RoR2'])
sns.distplot(back_test['RoR3'])
sns.distplot(back_test['RoR4'])
sns.distplot(back_test['kodex_RoR'])